# 📊 Football Player Analytics Pipeline
## Notebook 2: Data Processing (ALL FEATURES)

This notebook:
1. Loads ALL stat types (standard, shooting, passing, possession, gca, misc)
2. Merges them into one comprehensive dataset
3. Filters for forwards/attackers
4. Uses ALL available per-90 features for clustering

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Libraries loaded!")

## 1. Check Available Data Files

In [ ]:
# List all CSV files in raw directory
csv_files = list(RAW_DIR.glob("*.csv"))
print(f"📁 Found {len(csv_files)} CSV files:")
for f in sorted(csv_files):
    size_kb = f.stat().st_size / 1024
    print(f"  • {f.name} ({size_kb:.1f} KB)")

## 2. Load and Merge All Stat Types

In [ ]:
def clean_column_name(col):
    """Clean multi-level FBref column names"""
    col = str(col)
    # Remove 'Unnamed' prefix
    if 'Unnamed' in col:
        parts = col.split('_')
        return parts[-1] if len(parts) > 1 else col
    return col

def load_and_clean_csv(filepath):
    """Load CSV and clean column names"""
    df = pd.read_csv(filepath)
    df.columns = [clean_column_name(c) for c in df.columns]
    return df

# Check for all_leagues_master.csv (standard stats)
master_file = RAW_DIR / "all_leagues_master.csv"

if master_file.exists():
    df = load_and_clean_csv(master_file)
    print(f"✅ Loaded standard stats: {len(df)} players, {len(df.columns)} columns")
else:
    raise FileNotFoundError("Run Notebook 01 first to scrape data!")

In [ ]:
# Try to load additional stat types and merge
stat_files = {
    'shooting': RAW_DIR / "all_leagues_shooting.csv",
    'passing': RAW_DIR / "all_leagues_passing.csv",
    'possession': RAW_DIR / "all_leagues_possession.csv",
    'gca': RAW_DIR / "all_leagues_gca.csv",
    'misc': RAW_DIR / "all_leagues_misc.csv",
    'defense': RAW_DIR / "all_leagues_defense.csv"
}

# Find player column in main df
player_col = [c for c in df.columns if 'player' in c.lower()][0]
print(f"\nPlayer column: {player_col}")

for stat_name, filepath in stat_files.items():
    if filepath.exists():
        stat_df = load_and_clean_csv(filepath)
        stat_player_col = [c for c in stat_df.columns if 'player' in c.lower()][0]
        
        # Get new columns (not in main df)
        existing_cols = set(df.columns)
        new_cols = [c for c in stat_df.columns if c not in existing_cols]
        
        if new_cols:
            merge_cols = [stat_player_col] + new_cols
            stat_df_subset = stat_df[merge_cols].copy()
            stat_df_subset = stat_df_subset.rename(columns={stat_player_col: player_col})
            
            # Merge on player name
            df = df.merge(stat_df_subset, on=player_col, how='left')
            print(f"✅ Merged {stat_name}: +{len(new_cols)} new columns")
    else:
        print(f"⚠️ {stat_name}: file not found (run 01b to scrape)")

print(f"\n📊 Total columns after merge: {len(df.columns)}")

In [ ]:
# Show all columns
print("📋 All available columns:")
for i, col in enumerate(df.columns):
    print(f"  {i+1:2d}. {col}")

## 3. Filter for Forwards

In [ ]:
# Find position column
pos_col = [c for c in df.columns if 'pos' in c.lower()][0]
print(f"Position column: {pos_col}")
print(f"\nPosition distribution:")
print(df[pos_col].value_counts())

In [ ]:
# Filter for forwards (any position containing FW)
mask = df[pos_col].astype(str).str.upper().str.contains('FW', na=False)
forwards_df = df[mask].copy()

print(f"✅ Filtered: {len(df)} → {len(forwards_df)} forwards")

## 4. Filter for Minimum Minutes

In [ ]:
# Find minutes column
min_col = None
for col in forwards_df.columns:
    if 'min' in col.lower() and '90' not in col.lower():
        min_col = col
        break

if min_col:
    forwards_df[min_col] = pd.to_numeric(
        forwards_df[min_col].astype(str).str.replace(',', ''), 
        errors='coerce'
    )
    
    MIN_MINUTES = 450  # ~5 full games
    forwards_df = forwards_df[forwards_df[min_col] >= MIN_MINUTES]
    print(f"✅ After minimum {MIN_MINUTES} minutes: {len(forwards_df)} forwards")
else:
    print("⚠️ Could not find minutes column")

## 5. Identify ALL Per-90 Features

In [ ]:
# Find all per-90 columns (already normalized by playing time)
per90_cols = [c for c in forwards_df.columns if '90' in c.lower() or 'per90' in c.lower()]

print(f"📊 Found {len(per90_cols)} per-90 columns:")
for col in per90_cols:
    print(f"  • {col}")

In [ ]:
# Also find raw counting stats that we can convert to per-90
# Get 90s column for normalization
ninety_col = [c for c in forwards_df.columns if c.lower() == '90s' or '90s' in c.lower()][0]
forwards_df[ninety_col] = pd.to_numeric(forwards_df[ninety_col], errors='coerce')

print(f"\nUsing '{ninety_col}' for per-90 conversion")

In [ ]:
# Define which raw columns to convert to per-90
# These are counting stats that should be normalized
COUNTING_STATS = [
    'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'CrdY', 'CrdR',  # Performance
    'xG', 'npxG', 'xAG', 'npxG+xAG',  # Expected
    'PrgC', 'PrgP', 'PrgR',  # Progression
    'Sh', 'SoT', 'Dist',  # Shooting
    'KP', 'TB', 'Sw', 'Crs', 'CK',  # Passing
    'Att', 'Succ', 'Tkld',  # Dribbles/Carries
    'SCA', 'GCA',  # Goal/Shot creating actions
    'Won', 'Lost',  # Aerials
    'Fls', 'Fld',  # Fouls
]

# Convert each to per-90
converted_cols = []
for stat in COUNTING_STATS:
    # Find matching column (case-insensitive)
    matching = [c for c in forwards_df.columns if c.lower() == stat.lower() or c.endswith('_' + stat)]
    
    for col in matching:
        new_col = f"{col}_per90"
        if new_col not in forwards_df.columns and col not in per90_cols:
            forwards_df[col] = pd.to_numeric(forwards_df[col], errors='coerce')
            forwards_df[new_col] = forwards_df[col] / forwards_df[ninety_col].replace(0, np.nan)
            converted_cols.append(new_col)

print(f"\n✅ Converted {len(converted_cols)} stats to per-90:")
for col in converted_cols:
    print(f"  + {col}")

In [ ]:
# FINAL: Get ALL features for clustering
# Include existing per-90 columns + newly converted ones

all_per90_cols = per90_cols + converted_cols

# Remove duplicates and non-numeric columns
CLUSTERING_FEATURES = []
for col in all_per90_cols:
    if col in forwards_df.columns:
        # Check if numeric
        if pd.api.types.is_numeric_dtype(forwards_df[col]):
            if col not in CLUSTERING_FEATURES:
                CLUSTERING_FEATURES.append(col)

print(f"\n📊 FINAL CLUSTERING FEATURES ({len(CLUSTERING_FEATURES)}):")
for i, col in enumerate(CLUSTERING_FEATURES, 1):
    print(f"  {i:2d}. {col}")

## 6. Handle Missing Values

In [ ]:
# Check missing values
print("📊 Missing values in clustering features:")
for col in CLUSTERING_FEATURES:
    missing = forwards_df[col].isna().sum()
    pct = missing / len(forwards_df) * 100
    if pct > 0:
        print(f"  {col}: {missing} ({pct:.1f}%)")

In [ ]:
# Fill missing values with 0 (conservative - means they didn't do that action)
for col in CLUSTERING_FEATURES:
    forwards_df[col] = forwards_df[col].fillna(0)

print("✅ Filled missing values with 0")

## 7. Normalize Features

In [ ]:
# Apply MinMax scaling (0-1 range)
scaler = MinMaxScaler()

for col in CLUSTERING_FEATURES:
    norm_col = f"{col}_norm"
    forwards_df[norm_col] = scaler.fit_transform(forwards_df[[col]])

print(f"✅ Normalized {len(CLUSTERING_FEATURES)} features")

## 8. Preview Data

In [ ]:
# Show sample
id_cols = [player_col, 'Squad', pos_col, '_league'] if '_league' in forwards_df.columns else [player_col, 'Squad', pos_col]
id_cols = [c for c in id_cols if c in forwards_df.columns]

display_cols = id_cols + CLUSTERING_FEATURES[:8]  # First 8 features
print(f"📊 Sample data ({len(forwards_df)} forwards):")
display(forwards_df[display_cols].head(10))

In [ ]:
# Feature statistics
print("📊 Feature statistics:")
display(forwards_df[CLUSTERING_FEATURES].describe())

## 9. Save Processed Data

In [ ]:
# Save processed data
output_file = PROCESSED_DIR / "forwards_processed.csv"
forwards_df.to_csv(output_file, index=False)
print(f"💾 Saved: {output_file}")
print(f"   Rows: {len(forwards_df)}")
print(f"   Columns: {len(forwards_df.columns)}")

# Save feature list
feature_file = PROCESSED_DIR / "clustering_features.txt"
with open(feature_file, 'w') as f:
    f.write('\n'.join(CLUSTERING_FEATURES))
print(f"💾 Saved feature list: {feature_file}")
print(f"   {len(CLUSTERING_FEATURES)} features for clustering")

---
## ✅ Next: Run Notebook 03 for clustering

All available per-90 features will be used for clustering.